In [1]:
from ptgnn.runtime_config.run_config import fetch_loaders

In [2]:
data_config = {
    'dataset' : {
        'type' : "rs",
        'mask_chiral_tags': True,
        'transformation_mode': 'chienn_tree_basic',
    },
    'loader': {
        'general': {
            'n_neighbors_in_circle': 3,
            'batch_size': 32,
            'num_workers': 0,
        },
        'train': {},
        'test': {},
        'val': {}
    }
}

In [27]:
train_loader, _, val_loader = fetch_loaders(data_config=data_config)

In [4]:
for batch in val_loader:
    display(batch)
    break

DataBatch(x=[2356, 118], edge_index=[2, 6416], edge_attr=[6416, 80], pos=[2356, 6], y=[32], batch=[2356], ptr=[33], ptree=[2356])

# model for one type (Z)

In [5]:
# put through linear layers
# todo: do so only for a  few and not for all as not all needed for this module

In [7]:
k = 3
hidden_dim = 118

In [8]:
import torch

In [9]:
k_list = torch.nn.ModuleList([
    torch.nn.Linear(118, hidden_dim)
    for _ in range(k)
])

In [10]:
# put all elements through linear layers
# todo add masking to only do to relevant layers
elem_z_put_though = [
    k_elem(batch.x)
    for k_elem in k_list
]

In [11]:
elem_z_put_though

[tensor([[-0.1546, -0.3444,  0.0812,  ..., -0.1385, -0.0085,  0.3768],
         [-0.1090, -0.2817,  0.4002,  ..., -0.2364,  0.1774,  0.1820],
         [-0.0255, -0.3696,  0.3805,  ..., -0.3273, -0.0885,  0.1770],
         ...,
         [-0.2121, -0.2898,  0.0705,  ..., -0.1983, -0.0174,  0.2768],
         [ 0.0962, -0.1440,  0.2399,  ..., -0.1331,  0.0904,  0.2725],
         [-0.2121, -0.2898,  0.0705,  ..., -0.1983, -0.0174,  0.2768]],
        grad_fn=<AddmmBackward0>),
 tensor([[ 0.0263,  0.0888,  0.0928,  ..., -0.0615,  0.0965,  0.0922],
         [-0.0162,  0.3291, -0.0509,  ...,  0.3745,  0.1032, -0.0951],
         [ 0.2482,  0.2255, -0.0527,  ...,  0.2624,  0.0638, -0.1338],
         ...,
         [-0.0535,  0.2333,  0.0576,  ...,  0.1984,  0.0251, -0.1103],
         [ 0.0191,  0.2481,  0.1563,  ...,  0.3055,  0.3272, -0.1276],
         [-0.0535,  0.2333,  0.0576,  ...,  0.1984,  0.0251, -0.1103]],
        grad_fn=<AddmmBackward0>),
 tensor([[-0.1072, -0.0742, -0.3349,  ...,  0.22

In [12]:
import json

In [13]:
# extract permutation trees from string
permutation_trees = [
    json.loads(p_string)
    for p_string in batch.ptree
]

In [14]:
permutation_trees

[{'P': [1]},
 {'P': [0, {'Z': [3, 7, 5]}]},
 {'P': [3, {'Z': [0, 5, 7]}]},
 {'P': [2, {'Z': [9]}]},
 {'P': [5, {'Z': [0, 7, 3]}]},
 {'P': [4, {'Z': [14, 19]}]},
 {'P': [7, {'Z': [0, 3, 5]}]},
 {'P': [6]},
 {'P': [9, {'Z': [2]}]},
 {'P': [8, {'Z': [11, 13]}]},
 {'P': [11, {'Z': [8, 13]}]},
 {'P': [10, {'Z': [15, 17]}]},
 {'P': [13, {'Z': [8, 11]}]},
 {'P': [12]},
 {'P': [15, {'Z': [10, 17]}]},
 {'P': [14, {'Z': [4, 19]}]},
 {'P': [17, {'Z': [10, 15]}]},
 {'P': [16]},
 {'P': [19, {'Z': [4, 14]}]},
 {'P': [18, {'Z': [21, 23, 25]}]},
 {'P': [21, {'Z': [18, 25, 23]}]},
 {'P': [20, {'Z': [27, 31, 29]}]},
 {'P': [23, {'Z': [18, 21, 25]}]},
 {'P': [22, {'Z': [26, 35, 33]}]},
 {'P': [25, {'Z': [18, 23, 21]}]},
 {'P': [24]},
 {'P': [27, {'Z': [20, 29, 31]}]},
 {'P': [26, {'Z': [22, 33, 35]}]},
 {'P': [29, {'Z': [20, 31, 27]}]},
 {'P': [28]},
 {'P': [31, {'Z': [20, 27, 29]}]},
 {'P': [30]},
 {'P': [33, {'Z': [22, 35, 26]}]},
 {'P': [32]},
 {'P': [35, {'Z': [22, 26, 33]}]},
 {'P': [34]},
 {'P': [3

In [15]:
def depth(d):
    if isinstance(d, list):
        return max(map(depth, d))
    if isinstance(d, dict):
        return 1 + (max(map(depth, d.values())) if d else 0)
    return 0

In [16]:
# get number of layers
num_layers = depth(permutation_trees)
num_layers

2

In [17]:
def get_at_depth(d: object, depth: int = 1) -> list:
    if depth == 1:
        # return the values
        return list(d.values())
    else:
        return [
            *get_at_depth(child, depth-1)
            for child in d.values()
        ]

SyntaxError: iterable unpacking cannot be used in comprehension (1806935797.py, line 7)

In [ ]:
# get actions to take at level max
actions = [

    for tree in permutation_trees
]

In [18]:
def get_matrix(tree, depth, prefix: list = []) -> list:
    if isinstance(tree, list):
        return torch.cat([
            get_matrix(child, depth, prefix + [idx])
            for idx, child in enumerate(tree)
        ], dim=-1)

    elif isinstance(tree, dict):
        key = next(iter(tree.keys()))
        return get_matrix(tree[key], depth-1, prefix)

    elif isinstance(tree, int):
        return torch.tensor(prefix + [tree] + [-1]*depth)

    else:
        return [-1]

temp = get_matrix(permutation_trees, depth=num_layers)
temp

tensor([   0,    0,    1,  ...,    0, 2354,   -1])

In [19]:
def get_matrix(tree, depth, prefix: list = []) -> list:
    if isinstance(tree, list):
        return torch.cat([
            get_matrix(child, depth, prefix + [idx])
            for idx, child in enumerate(tree)
        ], dim=0)

    elif isinstance(tree, dict):
        key = next(iter(tree.keys()))
        return get_matrix(tree[key], depth-1, prefix)

    elif isinstance(tree, int):
        return torch.tensor(prefix + [tree] + [-1]*depth).reshape(1, -1)

    else:
        return [-1]

temp = get_matrix(permutation_trees, depth=num_layers)
temp[:10, :]

tensor([[ 0,  0,  1, -1],
        [ 1,  0,  0, -1],
        [ 1,  1,  0,  3],
        [ 1,  1,  1,  7],
        [ 1,  1,  2,  5],
        [ 2,  0,  3, -1],
        [ 2,  1,  0,  0],
        [ 2,  1,  1,  5],
        [ 2,  1,  2,  7],
        [ 3,  0,  2, -1]])

In [20]:
def get_matrix(tree, depth, prefix: list = []) -> list:
    if isinstance(tree, list):
        temp = [
            get_matrix(child, depth, prefix + [idx])
            for idx, child in enumerate(tree)
        ]
        return torch.cat(temp, dim=0)

    elif isinstance(tree, dict):
        key = next(iter(tree.keys()))
        return get_matrix(tree[key], depth-1, prefix)

    elif isinstance(tree, int):
        return torch.tensor(prefix + [tree] + [-1]*depth).reshape(1, -1)

    else:
        return [-1]

temp = get_matrix(permutation_trees, depth=num_layers)
temp[:10, :]

tensor([[ 0,  0,  1, -1],
        [ 1,  0,  0, -1],
        [ 1,  1,  0,  3],
        [ 1,  1,  1,  7],
        [ 1,  1,  2,  5],
        [ 2,  0,  3, -1],
        [ 2,  1,  0,  0],
        [ 2,  1,  1,  5],
        [ 2,  1,  2,  7],
        [ 3,  0,  2, -1]])

In [91]:
permutation_trees[1]

{'P': [0, {'Z': [3, 7, 5]}]}

In [86]:
torch.cat([torch.tensor([[1,2,3]]), torch.tensor([[1,2,3], [2,3,4],[5,6,7]])], dim=0)

tensor([[1, 2, 3],
        [1, 2, 3],
        [2, 3, 4],
        [5, 6, 7]])

In [77]:
selection = temp[0]
print(selection)
print(type(selection))
print(len(selection))

[1]
<class 'list'>
1


In [65]:
[-1]*0

[]

# simplistic version:

In [21]:
z_layer = torch.nn.ModuleList([
    torch.nn.Linear(118, 118)
    for _ in range(k)
])
z_final_layer = torch.nn.Linear(118,118)
z_elu = torch.nn.ELU()

In [22]:
p_layer = torch.nn.Linear(118, 118)
p_final_layer = torch.nn.Linear(118, 118)

In [23]:
def fetch_or_produce(subtree):
    if isinstance(subtree, int):
        return batch.x[subtree]
    else:
        return fulfill_module(subtree)

In [24]:
def fulfill_module(tree):
    # get node type
    node_type = next(iter(tree.keys()))

    # iterate over indices/trees and replace by data
    data_list = [
        fetch_or_produce(subtree)
        for subtree in tree[node_type]
    ]

    data_list = torch.stack(data_list)

    # put through layer
    if node_type == "P":
        # run through layer for each input
        after_p = p_layer(data_list)
        # sum up
        after_p = torch.sum(after_p, dim=0)
        # put through final linear layer
        after_p = p_final_layer(after_p)

        return after_p

    elif node_type == "Z":
        # run through embedding layers
        after_z = [
            layer(data_list)
            for layer in z_layer
        ]
        # shift
        _idx = torch.arange(data_list.shape[0])
        after_z = [
            embedding[torch.roll(_idx, shifts=idx), :]
            for idx, embedding in enumerate(after_z)
        ]
        after_z = torch.stack(after_z)
        # sum
        after_z = torch.sum(after_z, dim=0)
        # apply ELU
        after_z = z_elu(after_z)
        # layer
        after_z = z_final_layer(after_z)
        # sum
        after_z = torch.sum(after_z, dim=0)
        return after_z


In [25]:
torch.tensor([1,2,3,4])[torch.roll(torch.arange(4), shifts=1)]

tensor([4, 1, 2, 3])

In [26]:
out = torch.stack([
    fulfill_module(ptree)
    for ptree in permutation_trees
])
display(out)
display(out.shape)

tensor([[-0.0124, -0.0707,  0.0155,  ...,  0.1192,  0.0661,  0.0281],
        [ 0.2446,  0.5926, -0.0761,  ...,  0.1711,  0.2674, -0.1024],
        [ 0.1278,  0.5084, -0.0535,  ...,  0.2248,  0.2009, -0.0798],
        ...,
        [ 0.0330, -0.1159, -0.0019,  ...,  0.1173,  0.0836, -0.0365],
        [ 0.1646,  0.7595, -0.1034,  ...,  0.0929,  0.1940,  0.0537],
        [ 0.0330, -0.1159, -0.0019,  ...,  0.1173,  0.0836, -0.0365]],
       grad_fn=<StackBackward0>)

torch.Size([2356, 118])

In [97]:
new_node_values = []
for ptree in permutation_trees:
    new_node_values.append(
        fulfill_module(ptree)
    )

{'P': [1]}
{'P': [0, {'Z': [3, 7, 5]}]}
{'P': [3, {'Z': [0, 5, 7]}]}
{'P': [2, {'Z': [9]}]}
{'P': [5, {'Z': [0, 7, 3]}]}
{'P': [4, {'Z': [14, 19]}]}
{'P': [7, {'Z': [0, 3, 5]}]}
{'P': [6]}
{'P': [9, {'Z': [2]}]}
{'P': [8, {'Z': [11, 13]}]}
{'P': [11, {'Z': [8, 13]}]}
{'P': [10, {'Z': [15, 17]}]}
{'P': [13, {'Z': [8, 11]}]}
{'P': [12]}
{'P': [15, {'Z': [10, 17]}]}
{'P': [14, {'Z': [4, 19]}]}
{'P': [17, {'Z': [10, 15]}]}
{'P': [16]}
{'P': [19, {'Z': [4, 14]}]}
{'P': [18, {'Z': [21, 23, 25]}]}
{'P': [21, {'Z': [18, 25, 23]}]}
{'P': [20, {'Z': [27, 31, 29]}]}
{'P': [23, {'Z': [18, 21, 25]}]}
{'P': [22, {'Z': [26, 35, 33]}]}
{'P': [25, {'Z': [18, 23, 21]}]}
{'P': [24]}
{'P': [27, {'Z': [20, 29, 31]}]}
{'P': [26, {'Z': [22, 33, 35]}]}
{'P': [29, {'Z': [20, 31, 27]}]}
{'P': [28]}
{'P': [31, {'Z': [20, 27, 29]}]}
{'P': [30]}
{'P': [33, {'Z': [22, 35, 26]}]}
{'P': [32]}
{'P': [35, {'Z': [22, 26, 33]}]}
{'P': [34]}
{'P': [37]}
{'P': [36, {'Z': [39, 41, 43]}]}
{'P': [39, {'Z': [36, 43, 41]}]}
{'P

In [28]:
from tqdm import tqdm

In [31]:
device = 'cuda'
z_elu = z_elu.to(device)
z_layer = z_layer.to(device)
z_final_layer = z_final_layer.to(device)

p_layer = p_layer.to(device)
p_final_layer = p_final_layer.to(device)

for train_batch in tqdm(train_loader):
    # to device
    batch = batch.to(device)

    # extract permutation trees from string
    permutation_trees = [
        json.loads(p_string)
        for p_string in batch.ptree
    ]

    new_node_values = [
        fulfill_module(ptree)
        for ptree in permutation_trees
    ]

  6%|▌         | 107/1721 [01:32<23:21,  1.15it/s]


KeyboardInterrupt: 